In [ ]:
import pymysql.cursors
import pandas as pd
connection = pymysql.connect(host='ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com', 
                             user='read_only', password='WHC63fKyjudfSz2U', db='CAPC_APIGATEWAY',
                             cursorclass=pymysql.cursors.SSDictCursor)

In [ ]:
connection

In [ ]:
acc = []
for i in open('/home/trishalaswain/Documents/encDocIds','r').readlines():
    acc.append(i.rstrip('\n'))

In [ ]:
len(acc)

In [ ]:
# only pcs download by encDocIds
sql = """select dm.id,nlp_pcs_xml_result_s3_path from document_processing_detail dpd
inner join
document_mst dm ON dm.id = dpd.document_id
inner join
account_mst am ON am.id = dm.account_id
where dm.id in ({});""".format(",".join("'"+str(x)+"'" for x in acc))

In [ ]:
sql_query = pd.read_sql(sql,con= connection)

In [ ]:
sql_query

In [ ]:
cmd1 = []
for pcs_path,enc_id in zip(sql_query.nlp_pcs_xml_result_s3_path,sql_query.id):
    # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id)+"/"+str(enc_id) +"_pcs.xml")
    pcs_path = pcs_path.replace("'","\'")
    pcs_path = pcs_path.replace(" ","\ ")
    # pcs_path = pcs_path.replace("'","\'")
    cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id) +"_pcs.xml")

    # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(request_path)+"    "+str(enc_id)+"/"+ str(enc_id)+"_request.json")

In [ ]:
cmd1

In [ ]:
with open(r'/home/trishalaswain/Downloads/common-server/sh-files/testing.sh', 'w') as fp:
    for item in cmd1:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')